# An introduction to Artificial Neural Networks
Maciej Aleksandrowicz, MVG Group 2023, Machine Learning Laboratory, Version 4

## Glossary
| Acronym/Short name | Full name |
| --- | --- |
| ANN | Artificial Neural Network |
| Backprog | Backward propagation |

## Import packages
For today's classes you have to use only numpy! 

No `from framework import solution` is allowed.

In [ ]:
%pip install numpy
import numpy as np

## Part 1: Universal Aproximators
Artificial Neural Networks (ANN) can be seen as a functions aproximators, optimized on provided data. 

### Artificial Neuron Model
* It is slightly modified __McCulloch-Pitts Neuron (1943)__. 
    * instead of Mculloch-Pitts model, you can use any activation function, not just the original  proposition of **step function** with configurable threshold. 
* It would not be a huge mistake to call it colloquially as "**McCulloch-Pitts neuron model**" or just "**neuron model**". 
* Also known as **Sigma Neuron** (due to the sigma symbol for summation).
* It literaty works in 3 steps:
    1) Multiply **inputs** by their corresponding **weights**,
    2) **Add all products with** an offset term called **bias**,
    3) Pass the sum result through a "cramping" function called **activation function**.
        * For instance, __sigmoid(x)__ maps input to range (0,1).

### Matrix representation
Instead of writing equations for multiple neurons, we can arrange multiplication and summation operations into matrices.

### Example activation functions

### Example (deep) artificial neural network
Talking about "shallow" or "deep" ANN is tricky - it depends mainly on the context. More than 2-3 layers can be considered as "deep" network.

As you can wonder, the _Deep Learning_ subject is focused on ANNs with multiple layers.

### Forward propagation example

In [ ]:
import numpy as np
def ReLU(x):
    return np.maximum(0, x)

# Layer size
input_size = 2
neuron_units = 10

# Initialization
x = np.random.randn(input_size, 1)
weights = np.random.randn(neuron_units, input_size)
biases = np.random.randn(neuron_units, 1)

# Forward propagation
activation = np.dot(weights, x) + biases
y = ReLU(activation)

print(y)

### Gradient descent
* **Gradient Descent** is a method of optimizing (training) ANN's weights and biases.
    * There are other methods (ex. The Hebb's Rule) to train ANNs, but they are don't work as good as Gradient Descent
* It consist of 3 main steps:
    * **Forward Propagation** - to calculate output for comparison with desired output (i.e. to form a optimization problem with a Cost function).
    * **Backward Propagation** - to calculate derivatives of weights & biases (in respect to some Cost function).
    * **Update Parameters** - to (slightly) adjust weights & biases according to calculated derivatives.

#### Passing gradient to previous layer
* The **Cost function** $C$ is calculated **only once** for the output of the **whole network**.
    * NOTE: the cost function is also known as **loss function**.
* In general calculating $$\dfrac{\partial C}{\partial a^{(L-1)}}$$ will result in a vector of derivatives of all activations from previous layer.
* To calculate updates of weights & biases in previous layer $(L-1)$, it is necessary to use each element from the derivative vector $\dfrac{\partial C}{\partial a^{(L-1)}}$ for corersponding activations of neurons $a^{(L-1)}_1, a^{(L-1)}_2, \dots, a^{(L-1)}_m$ where $m$ is the size of $(L-1)$ layer.
* Then, just apply the chain rule further, adding more multiplication terms (like derivative of the activation function in the current layer).


#### Parameters update
```python
weights = weights - alpha * weights_derivative
bias = bias - alpha * bias_derivative
```
where `alpha` is a **learning rate** parameter which helps us not overshot the local minima during the optimization process of the weights & biases.

We are **subtracting** the calculated gradient (i.e. derivative) to descent into minimum.

### Gradient Descent implementation example
With:
* Forawrd Propagation
* Backward Propagation 
* Update Params

and definitions of 2 activation functions.

In [ ]:
'''
GOAL: Build a ANN network with 2 layers
* 1 layer:
    * Input size: 10
    * Output size, (aka. Neuron units; aka. layer size): 10
    * Activation function (for all units): ReLU
* 2 layer:
    * Input size: (same as previous layer size, here: 10)
    * Output size: 2
    * Activation function: Softmax
'''
import numpy as np
from typing import Tuple
from collections import namedtuple

# =============================== #

# Let's start with definitions of some activation functions
# ReLU with derivative for the 1st layer

def ReLU(Z):
    return np.maximum(0, Z)

def ReLU_deriv(Z):
    return Z > 0

# And Softmax for the 2nd layer 
# taken from https://stackoverflow.com/a/54977170

def softmax(Z):
    e = np.exp(Z)
    return e / np.sum(e, axis=1)

def softmax_deriv(Z):
    # Reshape the 1-d softmax to 2-d so that np.dot will do the matrix multiplication
    Z_reshaped = Z.reshape(-1,1)
    return np.diagflat(Z_reshaped) - np.dot(Z_reshaped, Z_reshaped.T)

# =============================== #

# Now, we need to somehow randomly initizalize weights & biases.
# Let's start with some preoparation, and define namedtuples for passing our network parameters
Layer = namedtuple('Layer', ['weights', 'bias'])
ANN = namedtuple('ANN', ['layer_1', 'layer_2'])

# Then, for simplicity, let's initialize our network in naive way
def init_ann_params(input_size: int, layer_1_size: int, layer_2_size: int) -> ANN:
    
    # The weights matrix must have dimensions equal to sizes of the input and the number of neural units
    weights_1 = np.random.rand(layer_1_size, input_size) - 0.5
    # The bias vector will have a one parameter (the bias) for each neural unit
    bias_1 = np.random.rand(layer_1_size, 1)- 0.5
    # Let's pack it into our namedtuple
    layer_1 = Layer(weights=weights_1, bias=bias_1)
    
    # Let's do it analogously with 2nd layer.
    # The input size must mach the size of the previous layer.
    # The output size of this layer will be also the output size of the whole network.
    weights_2 = np.random.rand(layer_2_size, layer_1_size)- 0.5
    bias_2 = np.random.rand(layer_2_size, 1)- 0.5
    layer_2 = Layer(weights=weights_2, bias=bias_2)
    
    # Finally, return our fresh network, packed in our namedtuple
    return ANN(layer_1, layer_2)

# =============================== #

# It's time for Forward Propagation definition
def forward_prop(ann: ANN, input_x: np.array) -> Tuple[np.array, np.array, np.array, np.array]:
    
    # Retrieve parameters for layer 1
    layer_1 = ann.layer_1
    # Using matrix notation multiply weights matrix by input vector and add bias vector for whole layer 1 
    sum_1 = layer_1.weights.dot( input_x ) + layer_1.bias
    # Then use the sum vector (each element consist of sum for each neuron in this layer) as input for activation function
    activate_1 = ReLU(sum_1)
    
    # Moving forward to the 2nd layer - retrieve parameters
    layer_2 = ann.layer_2
    # Use the output of the 1st layer as input for the matrix calcucations in 2nd layer
    sum_2 = layer_2.weights.dot( input_x ) + layer_2.bias
    # Use the sum vector with the activation function of the last layer
    activate_2 = softmax(sum_2)
    
    # Return the sums and the activations results
    # (The sums will be needed to calculate derivatives of the activation functions!)
    return sum_1, activate_1, sum_2, activate_2

# =============================== #
# After the forward propagation it's time for
# The creme de la creme of the implementation - Backward Propagation 

# For the input we will need:
#  1) sums and activations from the forward propagation,
#  2) the weights of the all previous layers
#  3) input sample with corresponding output sample (just one pair of samples!)
def backward_prop(sum_1: np.array, 
                  activate_1: np.array, 
                  sum_2: np.array, 
                  activate_2: np.array,
                  ann: ANN,
                  input_x: np.array, 
                  output_y: np.array) -> Tuple[np.array, np.array, np.array, np.array]:
    
    # First, calculate the derivative of the cost function
    # (Remember - it will always return a scalar value! )
    cost_deriv = 2 * np.sum(activate_2 - output_y)
    
    # Let's use it to calculate updates (derivatives) of the weights & biases in the last layer
    
    # Studying the equations of the backprog, we can observe, that the bias term is slightly easier to implement
    # What is more, *we can reuse it* in calculations of weights and the layer's input derivatives!
    # NOTE: The result is a vector!
    bias_2_deriv = softmax_deriv(sum_2) * cost_deriv
    
    # In addition, to obtain the derivative of the weights we need just to multiply the bias deriv by the layer's input!
    # NOTE: Using 2 vectors we want to obtain result with dimensions of weights matrix!
    weights_2_deriv = activate_1.dot( bias_2_deriv.transpose() )
    
    # Now, lest propagate the gradient to the first layer by calculating the derivative of the cost function in terms of the input
    layer_2 = ann.layer_2
    activate_1_deriv = layer_2.weights.dot( bias_2_deriv )
    
    # We can now calculate the derivatives for the 1st layer, reusing the above logic
    bias_1_deriv = ReLU_deriv(sum_1) * activate_1_deriv
    weights_1_deriv = input_x.dot( bias_1_deriv.transpose() )
    # NOTE: We don't want to calculate the derivative of the input of the wole network
    # (i.e. we don't want to pass the gradient to the dataset)
    
    # Return calculated updates (derivaites) for the weights and biases
    return weights_1_deriv, bias_1_deriv, weights_2_deriv, bias_2_deriv


# =============================== #

# Now lets define function for updating network parameters - weights & biases
# The alpha paramaeters is a learning rate
# Let's update the params in naive way for further simplification
def update_params(ann: ANN, 
                  weights_1_deriv: np.array, 
                  bias_1_deriv: np.array, 
                  weights_2_deriv: np.array, 
                  bias_2_deriv: np.array, 
                  alpha: float) -> ANN:
    layer_1 = ann.layer_1
    layer_1.weights = layer_1.weights - alpha * weights_1_deriv
    layer_1.bias = layer_1.bias - alpha * bias_1_deriv
    
    layer_2 = ann.layer_2
    layer_2.weights = layer_2.weights - alpha * weights_2_deriv
    layer_2.bias = layer_2.bias - alpha * bias_2_deriv
    
    # Return the ANN with updated weightes & biases
    return ANN(layer_1, layer_2)

# =============================== #

# Finally, let's define Gradient Descent to call above functions
def gradient_descent(input_x: np.array,
                     output_y: np.array, 
                     alpha: float, 
                     iterations: int) -> ANN:
    
    # First, initialize network with random parameters
    ann = init_ann_params()
    for _ in range(iterations):
        # 1) Forward Propragation
        sum_1, activate_1, sum_2, activate_2 = forward_prop(ann, input_x)
        # 2) Backwards Propagation
        weights_1_deriv, bias_1_deriv, weights_2_deriv, bias_2_deriv = backward_prop(sum_1,
                                                                                     activate_1, 
                                                                                     sum_2, 
                                                                                     activate_2, 
                                                                                     ann, 
                                                                                     input_x, 
                                                                                     output_y)
        # 3) Update parameters
        ann = update_params(ann, 
                            weights_1_deriv, 
                            bias_1_deriv, 
                            weights_2_deriv, 
                            bias_2_deriv,
                            alpha)
        
    # Return the updated network
    return ann

## Part 2: Basic implementation
In the following sections we will depend mainly on Numpy package

In [ ]:
import numpy as np
from typing import Callable

### Task 2.1
Implement an artificial neuron class, with **sigmoid** activation function. Use matrix operations (from Numpy package). Remember to define the activation function derivative. You can use following class-template or implement whole class by yourself.

In [ ]:
from typing import Callable, Tuple

def activation_function(x: float) -> float:
    return 1 / (1 + np.exp(-x))

def activation_function_deriv(x: float) -> float:
    sigmoid = activation_function(x)
    return sigmoid * (1 - sigmoid)

class Neuron:
    def __init__(self, input_size: int, act_func: Callable, act_func_deriv: Callable):
        self._init_weights_and_bias(input_size)
        self._activation_function = act_func
        self._activation_function_deriv = act_func_deriv
    
    def _init_weights_and_bias(self, input_size: int):
        self.weights = np.random.randn(input_size) - 0.5
        self.bias = np.random.randn() - 0.5
    
    def __call__(self, x: np.array) -> float:
        return self._forward_propagation(x)
    
    def _forward_propagation(self, x: np.array) -> float:
        z = np.dot(self.weights, x) + self.bias
        return self._activation_function(z)
    
    def gradient_descent(self, x: np.array, y_target: float, alpha: float, iterations: int) -> None:
        self.alpha = alpha
        for _ in range(iterations):
            self.weights_deriv, self.bias_deriv = self._backward_propagation(x, y_target)
            self._update_weights_and_bias()
    
    def _backward_propagation(self, x: np.array, y: float) -> Tuple[np.array, float]:
        # 1st - full forward propagation
        z = np.dot(self.weights, x) + self.bias
        a = self._activation_function(z)
        
        # 2nd - calculate the loss function (error)
        
        # 3rd - calculate derivatives based on forward propagation and the error
        dz = 2 * (a - y) * self._activation_function_deriv(z)
        dw = dz * x
        db = dz
        
        return dw, db
    
    def _update_weights_and_bias(self):
        self.weights = self.weights - self.alpha * self.weights_deriv
        self.bias = self.bias - self.alpha * self.bias_deriv

## Part 3: Artificial Neuron as binary clasifier
A single neuron used as binary classifier is also known as *perceptron*, frequently used as building block for *dense* layer. It can be used for logistic regression.

### Task 3.1
1) Using your Neuron class construct a following ANN:
  * Input size: 2
  * 1 layer with 1 unit with any activation function
  * Output size: 1

2) Perform separate trainings on provided datasets of truth tables of logic gates. You can experiment with number of iterations (start with n=500) and learnining rate (start with alpha = 0.1)

3) Visualize each dataset and ANN's result (a regression line, as function of two inputs).

4) Comment results

In [ ]:
import matplotlib.pyplot as plt

#### OR gate

In [ ]:
dataset_or_x = np.array([[0, 0], [0, 1], [1, 0], [1, 1]]).astype('float64')
dataset_or_y = [0, 1, 1, 1]

neuron = Neuron(
    input_size=2,
    act_func=activation_function,
    act_func_deriv=activation_function_deriv
)

alpha = 0.1
iterations = 500

for i in range(iterations):
    for x, y in zip(dataset_or_x, dataset_or_y):
        neuron.gradient_descent(x, y, alpha, 1)

predictions = [neuron(x) for x in dataset_or_x]

for x, prediction in zip(dataset_or_x, predictions):
    print(f"Input: {x}, Predicted Output: {prediction:.4f}")

In [ ]:
x_min, x_max = -0.5, 1.5
y_min, y_max = -0.5, 1.5
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01),
                     np.arange(y_min, y_max, 0.01))
grid = np.c_[xx.ravel(), yy.ravel()]

Z = np.array([neuron(x) for x in grid])
Z = Z.reshape(xx.shape)

plt.figure(figsize=(10, 6))
plt.contourf(xx, yy, Z, levels=[0, 0.5, 1], alpha=0.3, colors=['blue', 'orange'])
plt.scatter(dataset_or_x[:, 0], dataset_or_x[:, 1], c=dataset_or_y, edgecolor='k', s=100, marker='o', label='data points')
plt.title('OR gate: ANN predictions and decision boundary')
plt.xlabel('input 1')
plt.ylabel('input 2')
plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)
plt.axhline(0, color='black',linewidth=0.5, ls='--')
plt.axvline(0, color='black',linewidth=0.5, ls='--')
plt.grid()
plt.legend()
plt.show()

#### AND gate

In [ ]:
dataset_and_x = np.array([[0, 0], [0, 1], [1, 0], [1, 1]]).astype('float64')
dataset_and_y = [0, 0, 0, 1]

neuron = Neuron(
    input_size=2,
    act_func=activation_function,
    act_func_deriv=activation_function_deriv
)

alpha = 0.01
iterations = 10000

for i in range(iterations):
    for x, y in zip(dataset_and_x, dataset_and_y):
        neuron.gradient_descent(x, y, alpha, 1)

predictions = [neuron(x) for x in dataset_and_x]

for x, prediction in zip(dataset_and_x, predictions):
    print(f"Input: {x}, Predicted Output: {prediction:.4f}")

In [ ]:
x_min, x_max = -0.5, 1.5
y_min, y_max = -0.5, 1.5
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01),
                     np.arange(y_min, y_max, 0.01))
grid = np.c_[xx.ravel(), yy.ravel()]

Z = np.array([neuron(x) for x in grid])
Z = Z.reshape(xx.shape)

plt.figure(figsize=(10, 6))
plt.contourf(xx, yy, Z, levels=[0, 0.5, 1], alpha=0.3, colors=['blue', 'orange'])
plt.scatter(dataset_and_x[:, 0], dataset_and_x[:, 1], c=dataset_and_y, edgecolor='k', s=100, marker='o', label='data points')
plt.title('AND gate: ANN predictions and decision boundary')
plt.xlabel('input 1')
plt.ylabel('input 2')
plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)
plt.axhline(0, color='black',linewidth=0.5, ls='--')
plt.axvline(0, color='black',linewidth=0.5, ls='--')
plt.grid()
plt.legend()
plt.show()

#### XOR gate

In [ ]:
dataset_xor_x = np.array([[0, 0], [0, 1], [1, 0], [1, 1]]).astype('float64')
dataset_xor_y = [0, 1, 1, 0]

neuron = Neuron(
    input_size=2,
    act_func=activation_function,
    act_func_deriv=activation_function_deriv
)

alpha = 0.001
iterations = 1000

for i in range(iterations):
    for x, y in zip(dataset_xor_x, dataset_xor_y):
        neuron.gradient_descent(x, y, alpha, 1)

predictions = [neuron(x) for x in dataset_xor_x]

for x, prediction in zip(dataset_xor_x, predictions):
    print(f"Input: {x}, Predicted Output: {prediction:.4f}")

## Part 4: Multilayer perceptron
More neurons can be stacked together to model nonlinear properties.  

### Task 4.1
In this task you have to implement following ANN:
* Input size: 2
* 1 layer with 2 units with sigmoid activation function
* 1 layer with 1 unit with sigmoid activation function
* Output size: 1
    
Your Neuron class was not designed for ambitious merging of weights and biases during the gradient descent, nor for passing outputs to perform forward propagation. To overcome such inconvenience, please manually define dataflow and method calling for all Neurons. You can expand provided example.

In [ ]:
class NeuralNetwork():
    def __init__(self, input_size: int, act_func: Callable, act_func_deriv: Callable, alpha: 0.1):
        # TODO
        self._neuron_1 = Neuron(input_size, act_func, act_func_deriv)
        self._neuron_2 = Neuron(input_size, act_func, act_func_deriv)
        self._neuron_3 = Neuron(input_size, act_func, act_func_deriv)
        self.alpha = alpha
        
    def __call__(self, x: np.array) -> float:
        return self._network_forward_propagation(x)

    def _network_forward_propagation(self, x: np.array) -> float:
        a1 = self._neuron_1(x)
        a2 = self._neuron_2(x)
        input_3 = np.array([a1, a2])
        return self._neuron_3(input_3)

    def _network_backwards_propagation(self, x: np.array, y: np.array) -> None:
        # TODO
        # 1st - full forward propagation
        z1 = np.dot(self._neuron_1.weights, x) + self._neuron_1.bias
        a1 = self._neuron_1._activation_function(z1)

        z2 = np.dot(self._neuron_2.weights, x) + self._neuron_2.bias
        a2 = self._neuron_2._activation_function(z2)

        input_3 = np.array([a1, a2])
        
        z3 = np.dot(self._neuron_3.weights, input_3) + self._neuron_3.bias
        a3 = self._neuron_3._activation_function(z3)

        # 2nd - backward propagation

        error_3_deriv = 2 * (a3 - y) # deriv of MSE
        
        delta_3 = error_3_deriv * self._neuron_3._activation_function_deriv(z3)

        delta_2 = np.dot(delta_3, self._neuron_3.weights[1]) * self._neuron_2._activation_function_deriv(z2)

        delta_1 = np.dot(delta_3, self._neuron_3.weights[0]) * self._neuron_1._activation_function_deriv(z1)

        # 3rd calculate gradients
        dw3, db3 = delta_3 * input_3, delta_3
        dw2, db2 = delta_2 * x, delta_2
        dw1, db1 = delta_1 * x, delta_1

        return dw3, db3, dw2, db2, dw1, db1

    def gradient_descent(self, x: np.array, y: np.array) -> None:
        # TODO
        gradient_sums = {'dw3': 0, 'db3': 0, 'dw2': 0, 'db2': 0, 'dw1': 0, 'db1': 0}
        for x_i, y_i in zip(x, y):
            dw3, db3, dw2, db2, dw1, db1 = self._network_backwards_propagation(x_i, y_i)
            gradient_sums['dw3'] += dw3
            gradient_sums['db3'] += db3
            gradient_sums['dw2'] += dw2
            gradient_sums['db2'] += db2
            gradient_sums['dw1'] += dw1
            gradient_sums['db1'] += db1
        
        m = len(x)
        avg_gradients = {k: v / m for k, v in gradient_sums.items()}

        self._neuron_3.weights -= self.alpha * avg_gradients['dw3']
        self._neuron_3.bias -= self.alpha * avg_gradients['db3']

        self._neuron_2.weights -= self.alpha * avg_gradients['dw2']
        self._neuron_2.bias -= self.alpha * avg_gradients['db2']

        self._neuron_1.weights -= self.alpha * avg_gradients['dw1']
        self._neuron_1.bias -= self.alpha * avg_gradients['db1']

### Task 4.2
1) Train your ANN created in task 4.1 on the XOR dataset. You can experiment with number of iterations (start with n=500) and learning rate (start with alpha=0.1).

2) Visualize the dataset and ANN's result (a regression line, as function of two inputs).

3) Comment results.

In [ ]:
dataset_xor_x = np.array([[0, 0], [0, 1], [1, 0], [1, 1]]).astype('float64')
dataset_xor_y = [0, 1, 1, 0]

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
alpha = 0.1
iterations = 10000
nn = NeuralNetwork(input_size=2, act_func=activation_function, act_func_deriv=activation_function_deriv, alpha=alpha)

for _ in tqdm(range(iterations)):
    nn.gradient_descent(dataset_xor_x, dataset_xor_y)


predictions = [nn(x) for x in dataset_xor_x]

for x, prediction in zip(dataset_xor_x, predictions):
    print(f"Input: {x}, Predicted Output: {prediction:.4f}")

## Remarks
* **Do not implement ANN by yourself** - use already tested open-source frameworks with hardware acceleration, such as PyTorch, Keras, TensorFlow, Jax+Haiku, etc.
* **Every ANN is just an aproximator for a certain (often unknown) function** - Nothing more, nothing less. The learning procedure is data-based brutal force function derivation.
* **Despite current knowledge, selecting ANN dimensions is still more art than science** - every fixed parameter can be considered as "hyperparameter", which can be further optimized by an adequate algorithm.
* **Selecting an activation function is not trivial** - always consider dimishing gradient and calculation cost

## Further reading
* [YouTube - 3Blue1Brown - Backpropagation calculus](https://www.youtube.com/watch?v=tIeHLnjs5U8)
* [Brilliant - Perceptron](https://brilliant.org/wiki/perceptron/)
* [builtin - How Does Backpropagation in a Neural Network Work?](https://builtin.com/machine-learning/backpropagation-neural-network)
* [edureka! - Backpropagation – Algorithm For Training A Neural Network](https://www.edureka.co/blog/backpropagation/)
* [Visual backpropagation description](https://sebastianraschka.com/faq/docs/visual-backpropagation.html)